In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
!pip3 install text2num
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=f65ddc87d2b67f938b3bb51949ec02f34eaf5a366ba379ff2b768e536b05c70c
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [3]:
!pip install word2number

  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5566 sha256=95568fcc7c4919598078e6890657e074addff5b625ff00bda646f761c868ba95
  Stored in directory: /root/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number


In [4]:
from word2number import w2n

In [ ]:
##sympy setup
!git clone https://github.com/sympy/sympy.git
import os
os.chdir('sympy/')
os.getcwdb()
!python -m pip install -e .

In [5]:
import re
import pandas as pd
#from datasets import load_dataset
import time
import pickle
from tqdm import tqdm

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
#df=pd.read_pickle("/content/drive/MyDrive/NLP project/multitool_commons/data_3.pkl")
#df=pd.read_pickle("/content/drive/MyDrive/NLP project/abhishek2.pkl")
df=pd.read_pickle("/content/drive/MyDrive/NLP project/testing data/gemma1_syn_test.pkl")


In [41]:
df.tail()

,Q,A,W,C,test_prompt,sft_response
103,"How many top-40 hits did the singer of ""Don't ...",38,"Don't Forbid Me > ""Don't Forbid Me"" is a popul...",First search for [Pat Boone -Wiki-> y1]. Searc...,### INSTRUCTION\nYour task is to generate a ch...,C: Search [Don't Forbid Me artist -Wiki-> y1]....
104,"What year was the first movie, which starred t...",2008,Nick Jonas &amp; the Administration > Nick Jon...,First search for [Camp Rock -Wiki-> y1]. Searc...,### INSTRUCTION\nYour task is to generate a ch...,"C: First, search [Nick Jonas & the Administrat..."
105,What year was the leader of the Stanford marsh...,1930,Stanford marshmallow experiment > The Stanford...,First search for [Walter Mischel -Wiki-> y1]. ...,### INSTRUCTION\nYour task is to generate a ch...,C: First search for [y1 = when was the leader ...
106,How many acres larger is the Oakland Universit...,1443,Nadine Deleury > She continues teaching music...,First search for [Oakland University campus -W...,### INSTRUCTION\nYour task is to generate a ch...,C: First search [number of acres in the Oaklan...
107,How many years was the company founded before ...,0,Sony Ericsson K600 > The K608i was available ...,First search for [Hutchison 3G -Wiki-> y1]. No...,### INSTRUCTION\nYour task is to generate a ch...,C: First find out [The year in which the compa...


In [9]:
df.shape

(108, 6)

In [10]:
import requests
import json
from bs4 import BeautifulSoup
from pprint import pprint
import wikipedia

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')


def wikipedia_seach(query):
  language_code = 'en'

  number_of_results = 3
  headers = {
    # 'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
    'User-Agent': 'YOUR_APP_NAME (YOUR_EMAIL_OR_CONTACT_PAGE)'
  }

  base_url = 'https://api.wikimedia.org/core/v1/wikipedia/'
  endpoint = '/search/page'
  url = base_url + language_code + endpoint

  search_query = query
  parameters = {'q': search_query, 'limit': number_of_results}
  response = requests.get(url, headers=headers, params=parameters)
  response = json.loads(response.text)

  searched_titles=[]
  for page in range(len(response['pages'])):
    display_title = response['pages'][page]['title']
    searched_titles.append(display_title)
    response['pages'][page]["article_url"] = 'http://en.wikipedia.org/?curid=' + str(response['pages'][page]['id'])
    response['pages'][page]['excerpt'] = BeautifulSoup(response['pages'][page]['excerpt']).get_text() #cleaning excerpts and saving

  excerpt_text=search_query+": "+"\n".join([page['excerpt'] for page in response['pages']]) #excerpt text

  if(len(searched_titles)!=0):
    #go to page whose title+ excerpt is most similar
    # Encode search query
    query_embedding = sentence_model.encode(search_query)
    page_texts = [page['title'] + ": " + page['excerpt'] for page in response['pages']] #page title + excerpt
    page_embeddings = sentence_model.encode(page_texts)
    # Calculate similarity scores for each page
    similarities = cosine_similarity([query_embedding], page_embeddings)[0]

    title_score_dict = dict(zip(searched_titles, similarities))

    # Get the index of the page with the highest similarity
    max_similarity_index = similarities.argmax()

    # Output the page id with the highest similarity
    max_similarity_page_id = response['pages'][max_similarity_index]['id']

    try:
      a=wikipedia.page(pageid=max_similarity_page_id)
      full_content=a.content
      return title_score_dict,excerpt_text,full_content
    except:
      return {},"",""
  else:
    return {},"",""

def QA(ques, context):
  result = question_answerer(question=ques,context=context)
  return result['answer'], result['score']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [11]:
import requests
import json
import numpy as np
from text_to_num import text2num

url = "https://google.serper.dev/search"

def google_search(query):
  payload = json.dumps({
  "q": query
})
  headers = {
  'X-API-KEY': '0fe4c44f967d645dc68aaaac5df760a012cd6ce0',
  'Content-Type': 'application/json'
  }
  excerpt_text = ""
  full_content = ""
  most_similar_snippets = ""
  title_score_dict = {}


  try:
    response = requests.request("POST", url, headers=headers, data=payload)
    response = json.loads(response.text)
    #print(response)
    if 'answerBox' in response:
      if 'snippetHighlighted' in response['answerBox']:
        excerpt_text = response['answerBox']['snippetHighlighted'][0]
      elif 'answer' in response['answerBox']:
        excerpt_text = response['answerBox']['answer']
      else:
        excerpt_text = response['answerBox']['snippet']

    else:
      #print("HERE")
      full_content = [results['title'] + ": " + results['snippet'] for results in response['organic']]

    if full_content:
      query_embedding = sentence_model.encode(query)
      snippet_embeddings = sentence_model.encode(full_content)
      similarities = cosine_similarity([query_embedding], snippet_embeddings)[0]
      snippets_greater_than_70 = np.where(similarities > 0.50)[0]
      most_similar_snippets = " ".join([full_content[index] for index in snippets_greater_than_70])

    return title_score_dict,excerpt_text,most_similar_snippets
  except:
    return {},"",""

In [12]:
## CLEANING

In [ ]:
df[['question', 'answer','W']].rename(columns={'question': 'Q', 'answer': 'A'})

In [14]:
row=9
print(df.loc[row]['Q'])
print(df.loc[row]['C'])
print(df.loc[row]['A'])

Svengali is a supernatural drama/horror film, released in the same year as John Barrymore's birth year minus 11 years?
First search for [John Barrymore birth year -Wiki-> y1]. Now search for the year of film release [y1 -QA(When was Svengali film released ?)-> y2]. Now subtract 11 from it [y2 - 11 = y3]. The answer is y3.
1931


### EXPERIMENT

In [15]:
from sympy import *
import re

In [16]:
def process_list_tuple(rep): ## DECISION HERE
  if(str(type(rep[0]))!="<class 'tuple'>"): #if wikipedia article comes as in query, return empty string as it is anyways invalid
    return ""
  #for QA result, best score string returned
  non_empty_rep = [item for item in rep if item]  # Filter out empty tuples
  if non_empty_rep:
    rep = max(non_empty_rep, key=lambda x: x[1])[0]  # Get the string with the highest score
  else:
    rep = ""  # If all tuples are empty, return an empty string
  return str(rep)


def get_variables(text):
  matches = re.findall(r'\[(.*?)\]', text)
  variables={}
  wiki_searches=[]
  errors=[]
  variables['all_eqs']=[]

  def replace_variable(match):
    #print(variables)
    rep = variables.get(match.group(), match.group())
    if isinstance(rep, list):
        rep=process_list_tuple(rep)
    return rep

  for index, instruction in enumerate(matches):
    #print('at ', instruction)
    #print(variables)
    #---WIKI TOOL---
    if(instruction.find("-Wiki")!=-1):
      query,ans=instruction.split(" -Wiki-> ") #for in query variables
      in_query_var=re.search(r'y\d+', query)
      if(in_query_var):
        query=re.sub(r'y\d+', replace_variable, query)

      #title_score_dict,excerpt_text,full_content=google_search(query)
      #forcing wiki here
      #excerpt_text=''
      #full_content=''

      #get question of next tool in query
      next_inst=matches[index+1]
      google_query=query
      try:
        if(next_inst.find("-QA")!=-1):
          qq = re.search(r'\((.*?)\)', next_inst.split("->")[0])
          nextq = qq.group(1)
          if(nextq.lower().find(query.lower())!=-1):
            google_query=nextq
          else:
            google_query=query+" "+nextq
      except:
        print('error in combining')

      title_score_dict,excerpt_text,full_content=google_search(google_query)

      if not excerpt_text and not full_content:
        print('WIKI SEARCH')
        title_score_dict,excerpt_text,full_content=wikipedia_seach(query)#wikipedia search
        if title_score_dict: #SAVING IN VARIABLES HERE
          #print('HERE updating varibale of wiki answer',ans.strip())
          variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content of wiki
          wiki_searches.append(title_score_dict)
        else:
          errors.append(f"Error: Wikipedia search unsuccessful for query: '{query}'")
          break
      else:
        variables[ans.strip()] = [excerpt_text, full_content]#replaces the answer variable with extracted content of google

    #---QA TOOL---
    elif(instruction.find("-QA")!=-1):
      q,a=instruction.split("->")
      result = re.search(r'\((.*?)\)', q)
      if result:
          question = result.group(1)
      else:
          errors.append("Error in finding question")
          continue

      pre=q.split(" -QA")[0]
      context_exc=""
      full_context=""
      if (pre.find("+")!=-1): #if multiple contexts need to be appended

        vars=pre.split("+")
        for i in vars:
          if(i.strip() in variables.keys()):
            if((str(type(variables[i.strip()][0]))!="<class 'tuple'>")):
              context_exc=variables[i.strip()][0] #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=variables[i.strip()][1]
            else:
              t=process_list_tuple(variables[i.strip()])
              #print('-----t',t)
              context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
              full_context=t
          else:
            errors.append(f"Error: Context variable '{i.strip()}' not found")
            break

      else: #single context QA
        context_var=pre.strip()
        if context_var in variables.keys():
          if((str(type(variables[context_var][0]))!="<class 'tuple'>")):
            context_exc=variables[context_var][0] #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=variables[context_var][1]
          else:
            t=process_list_tuple(variables[context_var])
            context_exc=t #if variable came from wiki search, it would be a string else it would be a tuple
            full_context=t
        else:
          errors.append(f"Error: Context variable '{context_var}' not found")
          break

      #cant send empty context to QA model
      if(context_exc!=""):
        #print(question,context_exc)
        execerpt_run=QA(question, context_exc)
        if(execerpt_run[-1]<0.75 and len(full_context)>0):
          full_run=QA(question, full_context)
        else:
          full_run=()
      else:
        execerpt_run=()
        if(len(full_context)>0):
          full_run=QA(question, full_context)
        else:
          full_run=()


      #print('Updating variable',a.strip())
      variables[a.strip()]=[execerpt_run,full_run]

    #----MATH TOOL----
    else:
      #print('going to math tool')
      #print('at math',instruction)
      variables['all_eqs'].append(instruction)
      equation_variables = re.findall(r'y\d+', instruction)
      #print('found varibales',equation_variables)
      for var in equation_variables:
        if(var in variables.keys()):
          #print('on', var)
          var_ans=process_list_tuple(variables[var])
          #print('before process',variables[var])
          #print('on process',var_ans)
          numbers = re.findall(r'\d+(?:\.\d+)?', var_ans)#we try searching for numbers
          if(len(numbers)>0):
            #print('adding equation',var, numbers)
            #get year fro multi-tool syn
            choose=numbers[0]
            for i in numbers:
              if(len(i)==4):
                choose=i
            variables['all_eqs'].append(var+" = "+str(choose)) #making equation for previously retrieved numerical values ### DECISON HERE
            errors.append(f"WARNING: '{str(numbers)}'numbers in var ans")
          else:
            try:
              #var_ans=str(text2num(var_ans, "en"))
              var_ans=str(w2n.word_to_num(var_ans))
              numbers = re.findall(r'\d+(?:\.\d+)?', var_ans)#we try searching for numbers
              if(len(numbers)>0):
                #print('adding equation',var, numbers)
                variables['all_eqs'].append(var+" = "+str(numbers[0])) #making equation for previously retrieved numerical values ### DECISON HERE
                errors.append(f"WARNING: '{str(numbers)}'numbers in var ans")
            except:
              print('var ans is not a numeric',var_ans)
              errors.append(f"Error: '{var_ans}' not numeric")


  return variables,wiki_searches,errors


In [42]:
row=0
print(df.iloc[row]['Q'])
print(df.iloc[row]['C'])
print(df.iloc[row]['A'])
text=df.iloc[row]['sft_response']
matches = re.findall(r'\[(.*?)\]', text)
matches

The Dutch-Belgian television series that "House of Anubis" was based on first aired in how many years after 2000?
First search for [Het Huis Anubis first aired -Wiki-> y1]. First episode released in [y1 -QA(Which year was the first episode aired ?)-> y2]. Number of years it aired after 2000 [y2 - 2000 = y3]. The answer is y3.
6


['Dutch television series House of Anubis -Wiki-> y1',
 'y1 -QA(When did "House of Anubis" first air on Dutch television ?)-> y2']

In [37]:
v,w,e=get_variables(text)

In [38]:
v

{'all_eqs': ['y2 - 2000 = y3', 'y2 = 2011'],
 'y1': ['1 January 2011', ''],
 'y2': [('1 January 2011', 0.949802815914154), ()]}

In [ ]:
solve_all(v['all_eqs'])

In [21]:
def solve_all(equations):
  ans_dict={}
  list_eq=[]
  for i in equations:
    eq=sympify("Eq(" + i.replace("=", ",") + ")")
    list_eq.append(eq)
  #print(list_eq)
  result =solve(list_eq)
  #print(result)
  if(str(type(result))=="<class 'list'>"):
    result=result[0]
  for i in result.keys():
      ans_dict[str(i)]=result[i]
  return ans_dict

In [43]:
processed=[]

In [44]:

for row in tqdm(range(df.shape[0])):
  row_d=df.iloc[row].to_dict()
  try:
    answer_var=row_d['sft_response'].lower().split("answer is")[-1].replace(".","").strip()
    variables, wiki_searches, errors = get_variables(row_d['sft_response'])
    if(len(variables['all_eqs'])>0):
      #solve eqs
      try:
        variables.update(solve_all(variables['all_eqs']))
      except:
        errors.append("Equaltions not solvable")
    if(answer_var in variables.keys()):
      answer=variables[answer_var]
    else:
      answer="not found"
  except:
    variables=[]
    wiki_searches=[]
    errors=['Outer Error Handling']
    answer="not found"

  row_d['Variables'] = variables
  row_d['Searches'] = wiki_searches
  row_d['Errors'] = errors
  row_d['Generated_answer'] = answer
  processed.append(row_d)

  # Save to pickle file every 50 rows
  if row % 50 == 0 and row != 0:
      processed_df = pd.DataFrame(processed)
      #processed_df.to_pickle(f'processed_rows_wiki2.pkl')


# Save any remaining processed rows to a pickle file
if(processed):
    processed_df = pd.DataFrame(processed)
    #processed_df.to_pickle(f'processed_rows_final_wiki2.pkl')

  6%|▋         | 7/108 [00:06<01:26,  1.17it/s]

error in combining


  9%|▉         | 10/108 [00:08<01:28,  1.10it/s]

var ans is not a numeric 
var ans is not a numeric 


 12%|█▏        | 13/108 [00:13<02:08,  1.35s/it]

...

 91%|█████████ | 98/108 [02:28<00:15,  1.54s/it]

var ans is not a numeric 


 99%|█████████▉| 107/108 [02:40<00:01,  1.43s/it]

var ans is not a numeric 


100%|██████████| 108/108 [02:42<00:00,  1.50s/it]


In [45]:
processed_df.shape

(108, 10)

In [46]:
row=0
print(df.iloc[row]['Q'])
print(df.iloc[row]['C'])
print(df.iloc[row]['A'])
text=df.iloc[row]['sft_response']
matches = re.findall(r'\[(.*?)\]', text)
matches

The Dutch-Belgian television series that "House of Anubis" was based on first aired in how many years after 2000?
First search for [Het Huis Anubis first aired -Wiki-> y1]. First episode released in [y1 -QA(Which year was the first episode aired ?)-> y2]. Number of years it aired after 2000 [y2 - 2000 = y3]. The answer is y3.
6


['Dutch television series House of Anubis -Wiki-> y1',
 'y1 -QA(When did "House of Anubis" first air on Dutch television ?)-> y2']

In [47]:
processed_df.iloc[0]['Variables']

{'all_eqs': [],
 'y1': ['September 2006', ''],
 'y2': [('September 2006', 0.9778741002082825), ()]}

In [48]:
def check_answer(row):
  gen=row['Generated_answer']
  act=row['A']
  all_vars=row['Variables']
  #print(all_vars)

  if(gen==act):
    return True
  if(str(gen).find(str(act))!=-1):
    return True
  if(str(type(all_vars))=="<class 'dict'>"):
    for v in all_vars.values():
      if(str(v)==str(act)):
        return True

  return False

# Apply the function to each row
processed_df['is_final_in_solved'] = processed_df.apply(check_answer, axis=1)

In [49]:
processed_df['is_final_in_solved'].unique()

array([ True, False])

In [50]:
# Count the rows where final_answer is present in solved_answer list
count_valid = processed_df[processed_df['is_final_in_solved']].shape[0]

In [51]:
count_valid

27

In [ ]:
#50%

In [52]:
processed_df.head()

,Q,A,W,C,test_prompt,sft_response,Variables,Searches,Errors,Generated_answer,is_final_in_solved
0,"The Dutch-Belgian television series that ""Hous...",6,House of Anubis > House of Anubis is a mystery...,First search for [Het Huis Anubis first aired ...,### INSTRUCTION\nYour task is to generate a ch...,C: First search for [Dutch television series H...,"{'all_eqs': [], 'y1': ['September 2006', ''], ...",[],[],"[(September 2006, 0.9778741002082825), ()]",True
1,What would be the length of the track where th...,10.213,2013 Liqui Moly Bathurst 12 Hour > The 2013 Li...,First search for [Mount Panorama Circuit -Wiki...,### INSTRUCTION\nYour task is to generate a ch...,C: The length of the track where the 2013 Liqu...,"{'all_eqs': ['y3 - 4 = y4'], 'y3': y4 + 4}",[],[],not found,False
2,"The 1988 American comedy film, The Great Outdo...",4,The Great Outdoors (film) > The Great Outdoors...,First search for [Annette Bening -Wiki-> y1]....,### INSTRUCTION\nYour task is to generate a ch...,"C: First, find out [The 1988 American comedy f...","{'all_eqs': [], 'y1': ['The Great Outdoors (fi...",[],[],"[(), (Dec. 1, 2023, 0.9729866981506348)]",True
3,Handy Corner is an unincorporated Mississippi ...,133,"Handy Corner, Mississippi > Handy Corner is a ...","First search for [Olive Branch, Mississippi -W...",### INSTRUCTION\nYour task is to generate a ch...,C: The population of Handy Corner is [4 * popu...,[],[],[Outer Error Handling],not found,False
4,Dead Space 3 was released internationally 9 mo...,2013,Dead Space 2 > Dead Space 2 is a science ficti...,First search for [Dead Space 2 release -Wiki->...,### INSTRUCTION\nYour task is to generate a ch...,C: First search for [Dead Space 2 release date...,"{'all_eqs': ['y2 + 9 = y3', 'y2 = 2011'], 'y1'...",[],[WARNING: '['2011']'numbers in var ans],2020,False


In [39]:
processed_df.to_csv('/content/drive/MyDrive/NLP project/testing data/gemma1_syn_test_processed.csv')

In [93]:
processed_df.to_pickle("/content/drive/MyDrive/NLP project/multitool_commons/gpt_syn_multitool-1.pkl")

In [96]:
## hOW MANY ARE NON ERRORS

In [106]:
# Define the words to filter for
error_words = ['Error']

# Function to check if any word in a list is in the specified error_words
def has_error_words(lst):
    return not any(word in str(lst) for word in error_words)

# Apply the function to filter rows
non_error_df = processed_df[processed_df['Errors'].apply(has_error_words)]

In [108]:
non_error_df.shape

(144, 9)

In [109]:
solved_rows_df = processed_df[processed_df['is_final_in_solved'] == True]
solved_rows_df.shape

(51, 9)

In [110]:
# Identify rows in non_error_df that are not in solved_rows_df
unique_non_error_df = non_error_df[~non_error_df.index.isin(solved_rows_df.index)]

# Select 40 rows from the identified rows
selected_rows = unique_non_error_df.sample(n=40, random_state=42)  # Adjust the sampling criteria as needed

# Concatenate selected rows with solved_rows_df
concatenated_df = pd.concat([solved_rows_df, selected_rows])

In [111]:
concatenated_df.shape

(91, 9)

In [112]:
concatenated_df.head()

,Q,A,W,C,Variables,Searches,Errors,Generated_answer,is_final_in_solved
2,"The 1988 American comedy film, The Great Outdo...",4,The Great Outdoors (film) > The Great Outdoors...,First Search [Annette Bening -Wiki-> y1]. She ...,"{'all_eqs': ['2010 - y2 = y3', 'y2 = 2006'], '...",[],"[WARNING: '['10', '2006']'numbers in var ans]",4,True
3,"Dua Lipa, an English singer, songwriter and mo...",17,Dua Lipa > Her self-titled debut studio album...,First Search [Dua Lipa New Rules release -Wiki...,"{'all_eqs': ['y2 - 2000 = y3', 'y2 = 2017'], '...",[],"[WARNING: '['2016', '2017']'numbers in var ans]",17,True
5,How many years before 1947 was the creator of ...,6,"The Simpsons Theme > ""The Simpsons"" Theme"", al...",First find [Simpson's Theme creator -Wiki-> y1...,"{'all_eqs': ['1947 - y2 = y3', 'y2 = 1941'], '...",[],"[WARNING: '['28', '1941']'numbers in var ans]",6,True
6,The American Pre-Code comedy film featuring an...,7,Hat Check Girl > Hat Check Girl is a 1932 Amer...,First Search for [American Pre-Code comedy RKO...,"{'all_eqs': ['y2 - 1925 = y3', 'y2 = 1932'], '...",[],[WARNING: '['1932']'numbers in var ans],7,True
7,How many years after 1950 was the winner of th...,38,2016 Marrakesh ePrix > The 33-lap race was wo...,First search for [2016 Marrakesh ePrix winner ...,"{'all_eqs': ['y2 - 1950 = y3', 'y2 = 1988'], '...",[],"[WARNING: '['31', '1988']'numbers in var ans]",38,True


In [113]:
concatenated_df.to_pickle("/content/drive/MyDrive/NLP project/multitool_commons/atFinal-gpt_syn_multitool-1.pkl")